In [ ]:
!pip install pmaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 7.3 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 


In [ ]:
from pmaw import PushshiftAPI
import datetime as dt

start_time = int(dt.datetime(2020, 1, 1).timestamp())  
end_time = int(dt.datetime(2020, 7, 31).timestamp())   

api = PushshiftAPI()
posts = api.search_submissions(subreddit='spain',
                               after = start_time, 
                               before = end_time, 
                               q = '"coronavirus"||"covid"||"covid19"||"covid-19"||"corona virus"||"sars cov"||"sarscov"||"sars-cov-2"',
                               limit = None,
                               fields = ['id', 'author', 'created_utc', 'domain', 'url', 'title', 'num_comments', 'full_link', 'permalink'])
post_list = [post for post in posts]

In [ ]:
import pandas as pd
df_post = pd.json_normalize(post_list)
df_post['id'].shape

(1442,)

In [ ]:
from time import sleep
import requests

comments = []
i = 1
for post in df_post['id'].values:
  r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
  while r.status_code != 200:
    sleep(min((2**i), 60))
    r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
    i += 1

  i = 1
  comments.append(r.json())

In [ ]:
pd.json_normalize(pd.json_normalize(comments).explode('data')['data']).to_csv('espanha_multilanguage.csv')